# Deep learning 

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import tensorflow as tf


# Generate dummy data
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

from pathlib import Path
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os

In [ ]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
tf.test.gpu_device_name()

# Declaración de variables

In [ ]:
#organismo ="nematoda"
#organismo ="Globodera"
#organismo ="Heterodera"
#organismo ="Meloidogyne"
#organismo ="archaea"
organismo ="E_coli"

dataset = 1
nombre = ("ds" + str(dataset) + "_" + str(organismo))

r1 = ("Datos/resultados/" + str(organismo) + "/" + str(nombre))
r2 = (str(r1) + "/transformaciones/sin_atipicos/")
r3 = (str(r1) + "/deep_learning/")
resultado = (str(r3) + str(nombre))

t1="AAC"
t2="ACC_hidro"
t3="ACC_hidro_mass"
t4="ACC_mass"
t5="PseAAC_hidro"
t6="PseAAC_hidro_mass"
t7="PseAAC_mass"

#transformacion = [str(t1), str(t2), str(t3), str(t4), str(t5), str(t6),str(t7)]
transformacion = [str(t1), str(t2), str(t4), str(t5), str(t6),str(t7)]
Model_1 = [8,7,6,5,4,3,2]
Model_2 = [8,6,4,3,2]
Model_3 = [8,6,4,3]
Model_4 = [8,6,4]
Model_5 = [8,4]

Model = [Model_3,Model_4,Model_5]


In [ ]:
print("########## Deep learning " + str(nombre) +" ##########\n")

os.makedirs(str(r3), exist_ok=True)
with open(resultado, 'a') as f:
    print("########## Deep learning " + str(nombre) +" ##########\n",file=f)

for Mod in Model:
    print("########## Model " + str(Mod) +" ##########\n")

    with open(resultado, 'a') as f:
        print("########## Model " + str(Mod) +" ##########\n\n",file=f)    

    
    for transf in transformacion:
        titulo = (str(organismo) + " dataset "+ str(dataset)+" "+str(transf))
        csv = ("ds" + str(dataset) + "_" + str(transf) +"_"+ str(organismo))

        df = pd.read_csv(str(r2) + str(csv) + ".csv", header=None,prefix='X')
        count_row = df.shape[0]  # gives number of row count
        count_col = df.shape[1]  # gives number of col count



        seed = 9
        np.random.seed(seed)
        batch_size = 100
        num_classes = 2
        epochs = 10000    
        a = ( count_col - 1 )    
        input_dim = a 

        #convertir clases categoricas 'string' a discretas 'entero'
        df['class'] = pd.DataFrame(df['X' + str(a)].astype('category'))
        #df['class'] = df['class'].apply(lambda x: x.cat.codes)
        df['class'] =  pd.Categorical(df['class'])
        df['class'] =  df['class'].cat.codes
        Y = df['class'].values    
        del df['class']
        del df['X' + str(a)]
        X= df.values

        seed = 9
        np.random.seed(seed)

        # define 10-fold cross validation test harness
        kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

        #print(Y)

        cv_real = []
        cv_pred = []
        print (titulo)
        os.makedirs(str(r3), exist_ok=True)
        with open(resultado, 'a') as f:    
            print(titulo, file=f)
        for train_index, test_index in kfold.split(X, Y):
            train_index = tf.convert_to_tensor(train_index, dtype=tf.int64)

            #print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index] 

            ######################################### training

            model = Sequential()
            # create model
            model.add(Dense(6, activation='relu', input_dim=input_dim))
            #model.add(Dropout(0.5))
            for i in Mod:
                model.add(Dense(i, activation='relu'))
            
            #model.add(Dense(8, activation='relu'))
            #model.add(Dense(6, activation='relu'))
            #model.add(Dense(4, activation='relu'))
            #model.add(Dense(3, activation='relu'))
            #model.add(Dropout(0.5))
            model.add(Dense(num_classes, activation='softmax'))
            #print(model.summary())
            sgd = SGD(lr=0.001, momentum=0.09)
            #Adagrad = keras.optimizers.Adagrad(learning_rate=0.01)

            # Compile model
            model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

            y_train =  keras.utils.to_categorical(y_train, num_classes)
            y_test =  keras.utils.to_categorical(y_test, num_classes)

            # Fit the model
            history_callback  = model.fit(X_train, y_train, epochs=10000, batch_size=100, verbose=0)

            # evaluate the model
            y_pred = model.predict(X_test)

            #print('y_test: ',y_test)
            y_pred = np.argmax(y_pred,axis=1)
            y_test = np.argmax(y_test,axis=1)
            #print('y_pred: ',y_pred)

            #################################################3
            cv_real.extend(y_test)
            cv_pred.extend(y_pred)

            #learning metrics
            #cm = confusion_matrix(y_test, y_pred)
            #print ("Confusion Matrix")
            #print (cm)

            acc = accuracy_score(y_test, y_pred)
            with open(resultado, 'a') as f:   
                print ("Accuracy score: %f" % accuracy_score(y_test, y_pred), file=f)

            print ("Accuracy score: %f" % accuracy_score(y_test, y_pred))

        cm = confusion_matrix(cv_real[:], cv_pred[:])
        with open(resultado, 'a') as f:   
            print ("Confusion Matrix", file=f)
            print (cm, file=f)
            print("Var : ",np.var(cv_pred), file=f)
            print ("Accuracy score: %f" % accuracy_score(cv_real, cv_pred), file=f)
            print("---------------------------------------------------------------------<>\n\n\n", file=f)

        print ("Confusion Matrix")
        print (cm)
        print("Var : ",np.var(cv_pred))
        print ("Accuracy score: %f" % accuracy_score(cv_real, cv_pred))
        print("---------------------------------------------------------------------<>\n\n\n")